# Imports

In [ ]:

import Orange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import friedmanchisquare
from scipy.stats import wilcoxon

### Pegando os resultados de cada base de dados

In [ ]:

result_abalone = pd.read_csv('resultados/abalone.csv', index_col=0)
result_balance = pd.read_csv('resultados/balance.csv', index_col=0)
result_cogumelos = pd.read_csv('resultados/cogumelos.csv', index_col=0)
result_ecoli = pd.read_csv('resultados/ecoli.csv', index_col=0)
result_folhas = pd.read_csv('resultados/folhas.csv', index_col=0)
result_habermans = pd.read_csv('resultados/habermans.csv', index_col=0)
result_inflammations = pd.read_csv('resultados/inflammations.csv', index_col=0)

result_jogodavelha = pd.read_csv('resultados/jogodavelha.csv', index_col=0)
result_sementes = pd.read_csv('resultados/sementes.csv', index_col=0)
result_tae = pd.read_csv('resultados/tae.csv', index_col=0)



##### Filtrando apenas os dados do F-measure | F1

In [ ]:
fmeasure_index = 2

result_abalone = result_abalone.iloc[fmeasure_index:, :]
result_balance = result_balance.iloc[fmeasure_index:, :]
result_cogumelos = result_cogumelos.iloc[fmeasure_index:, :]
result_ecoli = result_ecoli.iloc[fmeasure_index:, :]
result_folhas = result_folhas.iloc[fmeasure_index:, :]
result_habermans = result_habermans.iloc[fmeasure_index:, :]
result_inflammations = result_inflammations.iloc[fmeasure_index:, :]
result_jogodavelha = result_jogodavelha.iloc[fmeasure_index:, :]
result_sementes = result_sementes.iloc[fmeasure_index:, :]
result_tae = result_tae.iloc[fmeasure_index:, :]


In [ ]:

import inspect

main_result = None

def only_fmeasure(df):
    fmeasure_index = 2
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    l = [k for k, v in callers_local_vars if v is df]
    new_index_name = l[0]
    obj = df.rename(index={'F1': new_index_name})
    return obj

  
main_result = only_fmeasure(result_abalone)
main_result = main_result.append(only_fmeasure(result_balance))
main_result = main_result.append(only_fmeasure(result_cogumelos))
main_result = main_result.append(only_fmeasure(result_ecoli))
main_result = main_result.append(only_fmeasure(result_folhas))
main_result = main_result.append(only_fmeasure(result_habermans))
main_result = main_result.append(only_fmeasure(result_inflammations))
main_result = main_result.append(only_fmeasure(result_jogodavelha))
main_result = main_result.append(only_fmeasure(result_sementes))
main_result = main_result.append(only_fmeasure(result_tae))

main_result.index.names = ['Datasets']
main_result

In [ ]:
# main_result.describe()

##### Calculando a os rankings de cada base de dados

In [ ]:
from scipy.stats import rankdata

# função para rankear cada linha de 1 a 5, empates são considerados valores iguais
def ranking(row):
    
    r_len = len(row.values[0])
    
    u, v = np.unique(row, return_inverse=True)
    temp_r = (np.cumsum(np.bincount(v)))[v]
    rank = []
    
    for e in temp_r:
        rank.append((r_len - e) + 1)

    return [rank]

i_bases = len(main_result.values)

ranked_result = main_result.copy()

for i in range(i_bases):
    row = ranked_result.iloc[[i]]
    temp = row.replace(row.values, ranking(row))
    ranked_result.iloc[[i]] = temp

ranked_result

In [ ]:
# main_result.describe()

##### Calculando a média dos rankings 

In [ ]:
media = []

for col in ranked_result.columns:
    media.append(np.median(ranked_result[col]))
media

##### Pegando os valores de cada algoritmo para gerar o P-value pelo teste de Friedman

In [ ]:
knn_scores = main_result["knn_scores"].values
svc_scores = main_result["svc_scores"].values
mlp_scores = main_result["mlp_scores"].values
gnb_scores = main_result["gnb_scores"].values
arvore_scores = main_result["arvore_scores"].values


stat, p_value = friedmanchisquare(knn_scores, svc_scores, mlp_scores, gnb_scores, arvore_scores)
p_value

##### NEMENYI E DIAGRAMA CD

In [ ]:
names = main_result.columns.values

avranks = media # RANKS MEDIOS

cd = Orange.evaluation.scoring.compute_CD(avranks=avranks, n=10, alpha='0.05', test='nemenyi')
cd

In [ ]:
Orange.evaluation.scoring.graph_ranks(avranks=avranks, names=names, cd=cd, filename="nemenyi-test")